# CCSD theory for a closed-shell reference

In this notebook we will use wicked to generate equations for the CCSD method

In [1]:
import wicked as w

import psi4
import forte
import forte.utils
from forte import forte_options
import numpy as np

In [2]:
w.reset_space()
w.add_space("o", "fermion", "occupied", ["i", "j", "k", "l", "m", "n"])
w.add_space("v", "fermion", "unoccupied", ["a", "b", "c", "d", "e", "f"])

Top = w.op("T", ["v+ o", "v+ v+ o o"])
Hop = w.op(
    "H",
   ["o+ o",
    "v+ v",
    "o+ v", 
    "v+ o",
    "o+ o+ o o",
    "o+ o+ v o",
    "o+ o+ v v",
    "o+ v+ o o",
    "o+ v+ v o",
    "o+ v+ v v",
    "v+ v+ o o",
    "v+ v+ v o",
    "v+ v+ v v",
    ],
)

In [6]:
wt = w.WickTheorem()
Hbar = w.bch_series(Hop,Top,4)
expr = wt.contract(w.rational(1), Hbar, 0, 4)
mbeq = expr.to_manybody_equation("R")

def generate_equation(mbeq, nocc, nvir):
    res_sym = f"R{'o' * nocc}{'v' * nvir}"
    code = [f"def evaluate_residual_{nocc}_{nvir}(H,T):",
        "    # contributions to the residual"]
    if nocc + nvir == 0:
        code.append("    R = 0.0")
    else:
        dims = ','.join(['nocc'] * nocc + ['nvir'] * nvir)
        code.append(f"    {res_sym} = np.zeros(({dims}))")
    for eq in mbeq["o" * nocc + "|" + "v" * nvir]:
        contraction = eq.compile("einsum")
        if res_sym in contraction:
            code.append(f'    {contraction}')
    code.append(f'    return {res_sym}')
    funct = '\n'.join(code)
    print(f'\n\n{funct}\n')
    return funct

energy_eq = generate_equation(mbeq, 0,0)
exec(energy_eq)
t1_eq = generate_equation(mbeq, 1,1)
exec(t1_eq)
t2_eq = generate_equation(mbeq, 2,2)
exec(t2_eq)



def evaluate_residual_0_0(H,T):
    # contributions to the residual
    R = 0.0
    R += 1.000000000 * np.einsum("ai,ia->",H["vo"],T["ov"])
    R += 0.500000000 * np.einsum("abij,ia,jb->",H["vvoo"],T["ov"],T["ov"])
    R += 0.250000000 * np.einsum("abij,ijab->",H["vvoo"],T["oovv"])
    return R



def evaluate_residual_1_1(H,T):
    # contributions to the residual
    Rov = np.zeros((nocc,nvir))
    Rov += 1.000000000 * np.einsum("ba,ib->ia",H["vv"],T["ov"])
    Rov += 1.000000000 * np.einsum("ia->ia",H["ov"])
    Rov += -1.000000000 * np.einsum("bj,ja,ib->ia",H["vo"],T["ov"],T["ov"])
    Rov += 1.000000000 * np.einsum("bj,ijab->ia",H["vo"],T["oovv"])
    Rov += -1.000000000 * np.einsum("ij,ja->ia",H["oo"],T["ov"])
    Rov += 1.000000000 * np.einsum("bcja,ic,jb->ia",H["vvov"],T["ov"],T["ov"])
    Rov += -0.500000000 * np.einsum("bcja,ijbc->ia",H["vvov"],T["oovv"])
    Rov += -1.000000000 * np.einsum("ibja,jb->ia",H["ovov"],T["ov"])
    Rov += 1.000000000 * np.einsum("bcjk,kc,ijab->ia

## Compute the Hartree–Fock and MP2 energy

In [7]:
# setup xyz geometry for linear H4
geometry = """
H 0.0 0.0 0.0
H 0.0 0.0 1.0
H 0.0 0.0 2.0
H 0.0 0.0 3.0
H 0.0 0.0 4.0
H 0.0 0.0 5.1
symmetry c1
"""

(Escf, psi4_wfn) = forte.utils.psi4_scf(geometry,
                                        basis='sto-3g',
                                        reference='rhf',
                                        options={'E_CONVERGENCE' : 1.e-12})

## Prepare integrals for Forte

In [8]:
# Define the orbital spaces
mo_spaces = {'RESTRICTED_DOCC': [3],'RESTRICTED_UOCC': [3]}

# pass Psi4 options to Forte
options = psi4.core.get_options()
options.set_current_module('FORTE')
forte_options.get_options_from_psi4(options)

# Grab the number of MOs per irrep
nmopi = psi4_wfn.nmopi()
# Grab the point group symbol (e.g. "C2V")
point_group = psi4_wfn.molecule().point_group().symbol()
# create a MOSpaceInfo object
mo_space_info = forte.make_mo_space_info_from_map(nmopi, point_group,mo_spaces, [])
# make a ForteIntegral object
ints = forte.make_ints_from_psi4(psi4_wfn, forte_options, mo_space_info)

## Define orbital spaces and dimensions

In [9]:
occmos = mo_space_info.corr_absolute_mo('RESTRICTED_DOCC')
virmos = mo_space_info.corr_absolute_mo('RESTRICTED_UOCC')
allmos = mo_space_info.corr_absolute_mo('CORRELATED')
nocc = 2 * len(occmos)
nvir = 2 * len(virmos)

## Build the Fock matrix and the zeroth-order Fock matrix

In [10]:
H = {'oo': forte.spinorbital_fock(ints,occmos, occmos,occmos),
     'vv': forte.spinorbital_fock(ints,virmos, virmos,occmos),
     'ov': forte.spinorbital_fock(ints,occmos, virmos,occmos),
     'vo': forte.spinorbital_fock(ints,occmos, virmos,occmos),     
     'oovv' : forte.spinorbital_tei(ints,occmos,occmos,virmos,virmos),
     'ooov' : forte.spinorbital_tei(ints,occmos,occmos,occmos,virmos),
     'vvvv' : forte.spinorbital_tei(ints,virmos,virmos,virmos,virmos),
     'vvoo' : forte.spinorbital_tei(ints,virmos,virmos,occmos,occmos),
     'ovov' : forte.spinorbital_tei(ints,occmos,virmos,occmos,virmos),
     'ovvv' : forte.spinorbital_tei(ints,occmos,virmos,virmos,virmos),
     'vvov' : forte.spinorbital_tei(ints,virmos,virmos,occmos,virmos),
     'ovoo' : forte.spinorbital_tei(ints,occmos,virmos,occmos,occmos),
     'oooo' : forte.spinorbital_tei(ints,occmos,occmos,occmos,occmos)}

## Build the MP denominators

In [11]:
fo = np.diag(H['oo'])
fv = np.diag(H['vv'])

D = {}

d1 = np.zeros((nocc,nvir))
for i in range(nocc):
    for a in range(nvir):
        si = i % 2
        sa = a % 2
        if si == sa:
            d1[i][a] = 1.0 / (fo[i] - fv[a])
D['ov'] = d1
            
                    
d2 = np.zeros((nocc,nocc,nvir,nvir))
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                si = i % 2
                sj = j % 2
                sa = a % 2
                sb = b % 2
                if si == sj == sa == sb:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])
                if si == sa and sj == sb and si != sj:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])
                if si == sb and sj == sa and si != sj:
                    d2[i][j][a][b] = 1.0 / (fo[i] + fo[j] - fv[a] - fv[b])                    
D['oovv'] = d2

In [12]:
# Compute the MP2 correlation energy
Emp2 = 0.0
for i in range(nocc):
    for j in range(nocc):
        for a in range(nvir):
            for b in range(nvir):
                Emp2 += 0.25 * H["oovv"][i][j][a][b] ** 2 / (fo[i] + fo[j] - fv[a] - fv[b])
print(f"MP2 corr. energy: {Emp2:.12f} Eh")

MP2 corr. energy: -0.066236921094 Eh


In [13]:
def antisymmetrize_residual_2_2(Roovv):
    # antisymmetrize the residual
    Roovv_anti = np.zeros((nocc,nocc,nvir,nvir))
    Roovv_anti += 0.25 * np.einsum("ijab->ijab",Roovv)
    Roovv_anti -= 0.25 * np.einsum("ijab->jiab",Roovv)
    Roovv_anti -= 0.25 * np.einsum("ijab->ijba",Roovv)
    Roovv_anti += 0.25 * np.einsum("ijab->jiba",Roovv)    
    return Roovv_anti

def update_amplitudes(T,R,d):
    T['ov'] += np.einsum("ia,ia->ia",R['ov'],D['ov'])
    T['oovv'] += np.einsum("ijab,ijab->ijab",R['oovv'],D['oovv'])

In [14]:
ref_CCSD  = -0.107582941213 # from psi4numpy (H6)

T = {}
T["ov"] = np.zeros((nocc,nvir))
T["oovv"] = np.zeros((nocc,nocc,nvir,nvir))

header = "Iter.    Corr. energy       |R|       "
print("-" * len(header))
print(header)
print("-" * len(header))

maxiter = 40
for i in range(maxiter):
    R = {}
    Ewicked = float(evaluate_residual_0_0(H,T))
    R['ov'] = evaluate_residual_1_1(H,T)
    Roovv = evaluate_residual_2_2(H,T)
    R['oovv'] = antisymmetrize_residual_2_2(Roovv)

    update_amplitudes(T,R,D)

    # check for convergence
    norm_R = np.sqrt(np.linalg.norm(R['ov'])**2 + np.linalg.norm(R['oovv'])**2)
    print(f"{i:3d}    {Ewicked:+.12f}  {norm_R:e}") 
    if norm_R < 1.0e-9:
        break
        
print("-" * len(header))    
print(f"CCSD correlation energy: {Ewicked:+.12f} [Eh]")
print(f"Error:                   {Ewicked - ref_CCSD:+.12e} [Eh]")

--------------------------------------
Iter.    Corr. energy       |R|       
--------------------------------------
  0    +0.000000000000  7.023516e-01
  1    -0.066236921094  2.703474e-01
  2    -0.088946436431  1.217901e-01
  3    -0.098716611030  6.159876e-02
  4    -0.103057285019  3.406897e-02
  5    -0.105151066198  2.036844e-02
  6    -0.106201235112  1.283723e-02
  7    -0.106763171400  8.324145e-03
  8    -0.107078043378  5.488428e-03
  9    -0.107263730832  3.648039e-03
 10    -0.107377120315  2.437192e-03
 11    -0.107448498137  1.632438e-03
 12    -0.107494313804  1.095330e-03
 13    -0.107524165846  7.356232e-04
 14    -0.107543800661  4.943600e-04
 15    -0.107556804218  3.323383e-04
 16    -0.107565453926  2.234690e-04
 17    -0.107571225679  1.502818e-04
 18    -0.107575085059  1.010713e-04
 19    -0.107577669584  6.797739e-05
 20    -0.107579402171  4.572026e-05
 21    -0.107580564534  3.075065e-05
 22    -0.107581344774  2.068232e-05
 23    -0.107581868731  1.391045